In [22]:
import pandas as pd
import numpy as np
import urllib.request
import folium
import matplotlib.pyplot as plt
import seaborn as sns
import reverse_geocoder as rg
import pprint
import geocoder
import multiprocessing as mp
import geopy
from geopy.geocoders import Nominatim
from geopy.point import Point
!pip install geopandas
import geopandas as gpd

In [17]:

valuation = pd.read_csv("Assessor_Historical_Secured_Property_Tax_Rolls.csv", low_memory=False)

In [34]:
val2 = valuation.dropna(subset = ['the_geom'])
val2.the_geom

95             POINT (-122.367656626348 37.711031780644)
96             POINT (-122.377197630074 37.710780097332)
97             POINT (-122.377457728422 37.711619808594)
98            POINT (-122.36841914661701 37.71112888287)
99           POINT (-122.37510574759702 37.714233963743)
                               ...                      
2666112        POINT (-122.469234215927 37.720116938309)
2666113      POINT (-122.46903919315598 37.711526516997)
2666114         POINT (-122.483524712308 37.70956434276)
2666115        POINT (-122.473845101411 37.734423011387)
2666116    POINT (-122.46321903048002 37.71387055028001)
Name: the_geom, Length: 2663991, dtype: object

In [53]:
val2["x"] = val2["the_geom"].str.split("\s+").str[1].str.replace("\(", "")
val2["y"] = val2["the_geom"].str.split("\s+").str[2].str.rstrip(")")

# Convert extracted values to numeric type
val2["x"] = pd.to_numeric(val2["x"])
val2["y"] = pd.to_numeric(val2["y"])
val2["x"] 

/var/folders/cb/b43gvslj4f7b_st3sl96fjy40000gn/T/ipykernel_31834/147988451.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  val2["x"] = val2["the_geom"].str.split("\s+").str[1].str.replace("\(", "")
/var/folders/cb/b43gvslj4f7b_st3sl96fjy40000gn/T/ipykernel_31834/147988451.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val2["x"] = val2["the_geom"].str.split("\s+").str[1].str.replace("\(", "")
/var/folders/cb/b43gvslj4f7b_st3sl96fjy40000gn/T/ipykernel_31834/147988451.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

95        -122.367657
96        -122.377198
97        -122.377458
98        -122.368419
99        -122.375106
              ...    
2666112   -122.469234
2666113   -122.469039
2666114   -122.483525
2666115   -122.473845
2666116   -122.463219
Name: x, Length: 2663991, dtype: float64

In [23]:
import geopandas

data_poly = gpd.read_file("San Francisco ZIP Codes.geojson")

In [55]:
gdf = gpd.GeoDataFrame(val2, geometry=gpd.points_from_xy(val2.x, val2.y))
joined_gdf = gpd.sjoin(gdf, data_poly, op='within')

/Users/SallyFang/miniforge3/envs/w207_final/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3373: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/var/folders/cb/b43gvslj4f7b_st3sl96fjy40000gn/T/ipykernel_31834/953716553.py:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joined_gdf = gpd.sjoin(gdf, data_poly, op='within')


In [56]:
joined_gdf.columns

Index(['Closed Roll Year', 'Property Location', 'Parcel Number', 'Block',
       'Lot', 'Volume Number', 'Use Code', 'Use Definition',
       'Property Class Code', 'Property Class Code Definition',
       'Year Property Built', 'Number of Bathrooms', 'Number of Bedrooms',
       'Number of Rooms', 'Number of Stories', 'Number of Units',
       'Zoning Code', 'Construction Type', 'Lot Depth', 'Lot Frontage',
       'Property Area', 'Basement Area', 'Lot Area', 'Lot Code',
       'Tax Rate Area Code', 'Percent of Ownership', 'Exemption Code',
       'Exemption Code Definition', 'Status Code', 'Misc Exemption Value',
       'Homeowner Exemption Value', 'Current Sales Date',
       'Assessed Fixtures Value', 'Assessed Improvement Value',
       'Assessed Land Value', 'Assessed Personal Property Value',
       'Assessor Neighborhood District', 'Assessor Neighborhood Code',
       'Assessor Neighborhood', 'Supervisor District', 'Analysis Neighborhood',
       'the_geom', 'Row ID', 'SF Find 

In [58]:
val_clean = joined_gdf[["Closed Roll Year", "id", "Assessed Improvement Value", "Assessed Land Value", "Assessed Personal Property Value",
"Year Property Built", "Number of Bathrooms","Number of Bedrooms", "Number of Rooms", "Number of Stories", "Number of Units", "Assessor Neighborhood District", "Assessor Neighborhood Code"]]

In [74]:
#data cleaning
#keep only data 2013 - 2019
#total assessed value = assessed personal property value + assessed improvement value
#drop rows without zipcode & 0 assessed total value
val_clean = val_clean[(val_clean["Closed Roll Year"] >= 2013) & (val_clean["Closed Roll Year"] <= 2019)]
val_clean["Total Assessed Value"] = val_clean["Assessed Land Value"] + val_clean["Assessed Improvement Value"]
val_clean = val_clean.rename(columns = {"id":"Zipcode"})
#check for invalid zipcodes -> none
val_clean["Zipcode"].value_counts()
#check to look at total assessed value, drop the rows wtih 0 total assessed value
val_clean["Total Assessed Value"].value_counts()
val_clean = val_clean[val_clean["Total Assessed Value"] != 0]
val_clean["Total Assessed Value"].value_counts()

,Closed Roll Year,Zipcode,Assessed Improvement Value,Assessed Land Value,Assessed Personal Property Value,Year Property Built,Number of Bathrooms,Number of Bedrooms,Number of Rooms,Number of Stories,Number of Units,Assessor Neighborhood District,Assessor Neighborhood Code,Total Assessed Value
174033,2016,94121,181379,181379,0,1980.0,1.0,1,3.0,1.0,1,1.0,1E,362758
208896,2016,94121,401392,577844,0,1914.0,6.0,9,21.0,3.0,6,1.0,1D,979236
208897,2016,94121,259562,389342,0,1925.0,2.0,4,8.0,1.0,1,1.0,1E,648904
210872,2016,94121,294460,441694,0,1911.0,1.0,3,6.0,2.0,1,1.0,1E,736154
211974,2016,94121,488640,1140160,0,1900.0,2.0,0,6.0,2.0,1,1.0,1A,1628800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2663725,2019,94134,220700,220700,0,1994.0,2.0,2,5.0,1.0,1,10.0,10B,441400
2663785,2019,94134,258092,258092,0,1990.0,2.0,2,4.0,1.0,1,10.0,10B,516184
2665189,2019,94134,134700,94328,0,1987.0,2.0,3,6.0,0.0,1,10.0,10B,229028
2665695,2019,94134,198114,198114,0,1987.0,2.0,3,6.0,0.0,1,10.0,10B,396228


In [75]:
val_clean.to_csv("val_clean.csv", index = False)